# Tile arithmetic
Rasterframes includes support for arithmetic operations across tiles, calling the function on the two cells that correspond to each other in tiles of the same size. This example uses localAdd, but localSubtract, localDivide, and localMultiply also exist. While not used in this example, localAddScalar, localSubtractScalar, etc can be uses to perform operations with a scalar, as opposed to the local values in another tile. 

Initial configs

In [ ]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._


implicit val spark = SparkSession.builder().
  master("local").appName("RasterFrames").
  config("spark.ui.enabled", "false").
  getOrCreate().
  withRasterFrames

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")

In [ ]:
val filenamePattern = "L8-B%d-Elkton-VA.tiff"
val bandNumbers = 1 to 4
val bandColNames = bandNumbers.map(b ⇒ s"band_$b").toArray

Once imported, join the tiles together based on a `spatialKey`.

In [ ]:
val joinedRF = bandNumbers.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ (b, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"band_$b") }.
  reduce(_ spatialJoin _)

New columns are added based on the results of these local functions. `localAdd` adds corresponding cells, `localDivide` divides them, `localSubtractScalar` subtracts a scalar from every cell in the input tile, and `localMultiplyScalar` multiplies each cell by a certain value.

In [ ]:
val addRF = joinedRF.withColumn("B1+B2", localAdd($"band_1", $"band_2")).asRF
val divideRF = joinedRF.withColumn("B1/B2", localDivide($"band_1", $"band_2")).asRF
val subScalarRF = joinedRF.withColumn("B1-100", localSubtractScalar($"band_1", 100)).asRF
val multScalarRF = joinedRF.withColumn("B1*2", localMultiplyScalar($"band_1", 2)).asRF

`tileMax` computes the max values of the cells in a tile, `tileSum` computes the sum of all cells, `tileMean` finds the mean value, and `tileMin` finds the minimum value.

In [50]:
addRF.select(tileMax($"B1+B2"), tileMax($"band_1"), tileMax($"band_2")).show()

+--------------+---------------+---------------+
|tileMax(B1+B2)|tileMax(band_1)|tileMax(band_2)|
+--------------+---------------+---------------+
|       42925.0|        20778.0|        22147.0|
+--------------+---------------+---------------+



In [51]:
divideRF.select(tileSum($"B1/B2"), tileSum($"band_1"), tileSum($"band_2")).show(1)

+--------------+---------------+---------------+
|tileSum(B1/B2)|tileSum(band_1)|tileSum(band_2)|
+--------------+---------------+---------------+
|       31389.0|   3.09149454E8|     2.806676E8|
+--------------+---------------+---------------+



In [52]:
subScalarRF.select(tileMean($"B1-100"), tileMean($"band_1")).show()

+-----------------+-----------------+
| tileMean(B1-100)| tileMean(band_1)|
+-----------------+-----------------+
|9734.874785264363|9834.874785264363|
+-----------------+-----------------+



In [53]:
multScalarRF.select(tileMin($"B1*2"), tileMin($"band_1")).show()

+-------------+---------------+
|tileMin(B1*2)|tileMin(band_1)|
+-------------+---------------+
|      18522.0|         9261.0|
+-------------+---------------+

